## Contexto do problema

Instituições financeiras liram diariamente com milhares de operações internas e transações que, quando mal monitoradas, podem gerar **risco operacional**.

Risco operacional envolve falhas em processos, sistemas, pessoas ou eventos externos, podendo resultar em:
- perdas financeiras
- falhas de compliance 
- impactos reputacionais

## Cenário simulado 

Neste projeto, será simulada uma base de dados contendo registros operacionais de uma instituição financeira fictícia 

Cada registro representa uma ocorrência operacional associada a:
- área responsável
- tipo de evento 
- data
- valor financeiro impactado 
- canal de origem 

O objetivo é identificar padrões que possam indicar **aumento de risco operacional**.

## Estrutura dos Dados

Cada linha do dataset representa uma ocorrência de risco operacional registrada pela instituição

A seguir estão descritas as principais variáveis consideradas na análise.

## Variáveis do Dataset 

- **id_ocorrencia**: identificador único da ocorrência
- **data_ocorrencia**: data em que o evento foi registrado 
- **area_responsavel**: área interna responsável pelo processo
- **tipo_evento**: classificação do evento operacional 
- **canal_origem**: canal onde o evento ocorreu (ex: digital, agência, backoffice)
- **valor_impacto**: valor financeiro estimado do impacto
- **nivel_risco**: classificação do risco (baixo, médio, alto)

In [ ]:
import pandas as pd

In [ ]:
colunas = [
    "impacto_financeiro",
    "impacto_cliente",
    "tempo_indisponibilidade",
    "frequencia_evento",
    "criticidade_sistema",
    "nivel_risco"
]

df = pd.DataFrame(columns=colunas)

df

In [ ]:
evento_baixo_1 = {
    "impacto_financeiro": 5000,
    "impacto_cliente": 0,
    "tempo_indisponibilidade": 0.5,
    "frequencia_evento": 1,
    "criticidade_sistema": 1,
    "nivel_risco": "baixo"
}

In [ ]:
evento_baixo_2 = {
    "impacto_financeiro": 8000,
    "impacto_cliente": 1,
    "tempo_indisponibilidade": 1.0,
    "frequencia_evento": 1,
    "criticidade_sistema": 2,
    "nivel_risco": "baixo"
}

evento_baixo_3 = {
    "impacto_financeiro": 3000,
    "impacto_cliente": 0,
    "tempo_indisponibilidade": 0.2,
    "frequencia_evento": 2,
    "criticidade_sistema": 1,
    "nivel_risco": "baixo"
}

df = pd.concat(
    [df, pd.DataFrame([evento_baixo_2, evento_baixo_3])],
    ignore_index=True
)

df

In [ ]:
evento_medio_1 = {
    "impacto_financeiro": 80000,
    "impacto_cliente": 1,
    "falha_processo": 1,
    "fraude_interna": 0,
    "recorrencia": 1
}

evento_medio_2 = {
    "impacto_financeiro": 100000,
    "impacto_cliente": 0,
    "falha_processo": 1,
    "fraude_interna": 0,
    "recorrencia": 1
}


In [ ]:
def classificar_risco(evento):
    if evento["impacto_financeiro"] < 50000:
        return "baixo"
    elif evento["impacto_financeiro"] < 150000:
        return "medio"
    else:
        return "alto"

In [ ]:
evento_medio_1["nivel_risco"] = classificar_risco(evento_medio_1)
evento_medio_2["nivel_risco"] = classificar_risco(evento_medio_2)

In [ ]:
evento_alto_1 = {
    "impacto_financeiro": 300000,
    "impacto_cliente": 3,
    "falha_processo": 1,
    "fraude_interna": 1,
    "recorrencia": 2
}

evento_alto_2 = {
    "impacto_financeiro": 500000,
    "impacto_cliente": 4,
    "falha_processo": 1,
    "fraude_interna": 1,
    "recorrencia": 3
}

In [ ]:
evento_alto_1["nivel_risco"] = classificar_risco(evento_alto_1)
evento_alto_2["nivel_risco"] = classificar_risco(evento_alto_2)

In [ ]:
df = pd.concat(
    [df, pd.DataFrame([evento_alto_1, evento_alto_2])],
    ignore_index=True
)

df

In [ ]:
df = pd.DataFrame([
    evento_baixo_1,
    evento_baixo_2,
    evento_baixo_3,
    evento_medio_1,
    evento_medio_2,
    evento_alto_1,
    evento_alto_2
])

df

In [ ]:
df["nivel_risco"].value_counts()

In [ ]:
x = df.drop(columns=["nivel_risco"])
y = df["nivel_risco"]

In [ ]:
x.dtypes 

In [ ]:
import random

def gerar_evento():
    impacto = random.randint(1000, 500000)

    evento = {
        "impacto_financeiro": impacto,
        "impacto_cliente": random.randint(0, 1),
        "tempo_indisponibilidade": round(random.uniform(0.1, 8), 2),
        "frequencia_evento": random.randint(1, 10),
        "criticidade_sistema": random.randint(1, 5),
        "falha_processo": random.randint(0, 1),
        "fraude_interna": random.randint(0, 1),
        "recorrencia": random.randint(1, 5),
    }
    if impacto < 50000:
     evento["nivel_risco"] = "baixo"
    elif impacto < 150000:
     evento["nivel_risco"] = "medio"
    else: 
     evento["nivel_risco"] = "alto"
    return evento

In [60]:
eventos = [gerar_evento() for _ in range(5000)]
df = pd.DataFrame(eventos)
df["nivel_risco"].value_counts()

nivel_risco
alto     3512
medio     961
baixo     527
Name: count, dtype: int64

In [ ]:
contagem = df["nivel_risco"].value_counts()
percentual = df["nivel_risco"].value_counts(normalize=True) * 100

resultado = pd.DataFrame({
    "quantidade": contagem,
    "percentual (%)": percentual.round(2)
})

resultado